<a href="https://colab.research.google.com/github/ChahineBA/Solar-Energy-Analysis/blob/main/Solar_Enerygy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pvlib

In [ ]:
pip install ephem

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from pvlib.location import Location

# Create a Location object for Tunisia
latitude = 36.8065
longitude = 10.1815
tz_str = 'Africa/Tunis'
site = Location(latitude, longitude, tz=tz_str)

# Create a DatetimeIndex for a specific date and time range
start_date = pd.Timestamp('2022-01-01 00:00')
end_date = pd.Timestamp('2022-12-31 23:30')
times = pd.date_range(start=start_date, end=end_date, freq='30min')
times = times[(times.time >= pd.Timestamp('00:00').time()) & (times.time <= pd.Timestamp('18:30').time())]

# Calculate the solar position for each time in the DatetimeIndex
solar_position = site.get_solarposition(times)

# Print the solar position DataFrame
print(solar_position)


                     apparent_zenith      zenith  apparent_elevation  \
2022-01-01 00:00:00       164.026921  164.026921          -74.026921   
2022-01-01 00:30:00       159.973996  159.973996          -69.973996   
2022-01-01 01:00:00       154.907512  154.907512          -64.907512   
2022-01-01 01:30:00       149.342600  149.342600          -59.342600   
2022-01-01 02:00:00       143.526748  143.526748          -53.526748   
...                              ...         ...                 ...   
2022-12-31 16:30:00        93.901817   93.901817           -3.901817   
2022-12-31 17:00:00        99.385949   99.385949           -9.385949   
2022-12-31 17:30:00       105.031360  105.031360          -15.031360   
2022-12-31 18:00:00       110.805192  110.805192          -20.805192   
2022-12-31 18:30:00       116.678061  116.678061          -26.678061   

                     elevation     azimuth  equation_of_time  
2022-01-01 00:00:00 -74.026921   32.936097         -3.297910  
2022-01-0

In [ ]:
from pvlib import irradiance
# Calculate the clear-sky irradiance for each time in the DatetimeIndex
irrad_data = site.get_clearsky(times)

# Define the surface orientation
surface_tilt = 20  # tilt angle in degrees
surface_azimuth = 180  # azimuth angle in degrees (south-facing)
irrad_data = site.get_clearsky(times)
# Calculate the POA irradiance for each time in the DatetimeIndex
poa_irrad = irradiance.get_total_irradiance(surface_tilt, surface_azimuth,
                                            solar_zenith=solar_position['apparent_zenith'],
                                            solar_azimuth=solar_position['azimuth'],
                                            dni=irrad_data['dni'], ghi=irrad_data['ghi'], dhi=irrad_data['dhi'])
print(poa_irrad)


                     poa_global  poa_direct  poa_diffuse  poa_sky_diffuse  \
2022-01-01 00:00:00         0.0         0.0          0.0              0.0   
2022-01-01 00:30:00         0.0         0.0          0.0              0.0   
2022-01-01 01:00:00         0.0         0.0          0.0              0.0   
2022-01-01 01:30:00         0.0         0.0          0.0              0.0   
2022-01-01 02:00:00         0.0         0.0          0.0              0.0   
...                         ...         ...          ...              ...   
2022-12-31 16:30:00         0.0         0.0          0.0              0.0   
2022-12-31 17:00:00         0.0         0.0          0.0              0.0   
2022-12-31 17:30:00         0.0         0.0          0.0              0.0   
2022-12-31 18:00:00         0.0         0.0          0.0              0.0   
2022-12-31 18:30:00         0.0         0.0          0.0              0.0   

                     poa_ground_diffuse  
2022-01-01 00:00:00              

In [ ]:
# Create a Figure object for the POA irradiance
fig = go.Figure()
fig.add_trace(go.Scatter(x=poa_irrad.index, y=poa_irrad['poa_global'], mode='lines', name='POA Irradiance'))
fig.update_layout(title_text="Plane of Array Irradiance")
fig.update_xaxes(title_text="Time")
fig.update_yaxes(title_text="Irradiance (W/m2)")

# Show the Figure object
fig.show()

In [ ]:
daily_gii = poa_irrad['poa_global'].resample('D').sum() / 1000  # convert W/m2 to kWh/m2
print(daily_gii)

2022-01-01    7.855169
2022-01-02    7.855222
2022-01-03    7.857619
2022-01-04    7.862336
2022-01-05    7.869348
                ...   
2022-12-27    7.887275
2022-12-28    7.874769
2022-12-29    7.864693
2022-12-30    7.857036
2022-12-31    7.851784
Freq: D, Name: poa_global, Length: 365, dtype: float64


In [ ]:
# Calculate the monthly solar irradiation by resampling the daily solar irradiation data by month and summing the values
monthly_gii = daily_gii.resample('M').sum()
print(monthly_gii)

2022-01-31    256.652275
2022-02-28    292.442063
2022-03-31    400.497598
2022-04-30    425.827817
2022-05-31    467.979083
2022-06-30    460.503174
2022-07-31    464.390911
2022-08-31    457.350247
2022-09-30    387.530605
2022-10-31    346.503876
2022-11-30    270.207975
2022-12-31    251.038590
Freq: M, Name: poa_global, dtype: float64


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Assume that `poa_irrad` is a DataFrame that contains the POA irradiance data
# with a DatetimeIndex and a column named 'poa_global'

# Calculate the daily solar irradiation by resampling the POA irradiance data by day and summing the values
daily_gii = poa_irrad['poa_global'].resample('D').sum() / 1000  # convert W/m2 to kWh/m2

# Calculate the monthly solar irradiation by resampling the daily solar irradiation data by month and summing the values
monthly_gii = daily_gii.resample('M').sum()

# Create a Figure object for the daily solar irradiation
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=daily_gii.index, y=daily_gii, mode='lines', name='Daily Solar Irradiation'))
fig1.update_layout(title_text="Daily Solar Irradiation (kWh/m2)")
fig1.update_xaxes(title_text="Date")
fig1.update_yaxes(title_text="Irradiation")

# Create a Figure object for the monthly solar irradiation
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=monthly_gii.index, y=monthly_gii, mode='lines', name='Monthly Solar Irradiation'))
fig2.update_layout(title_text="Monthly Solar Irradiation (kWh/m2)")
fig2.update_xaxes(title_text="Month")
fig2.update_yaxes(title_text="Irradiation")

# Show the Figure objects
fig1.show()
fig2.show()


In [ ]:
# Define the module efficiency and power rating
module_efficiency = 0.2  # module efficiency (fraction)
pr = 250  # power rating of the module (W)

# Calculate the hourly production of the photovoltaic system
hourly_production = (poa_irrad['poa_global']/1000) * module_efficiency * pr

# Create a Figure object for the hourly production
fig = go.Figure()
fig.add_trace(go.Scatter(x=hourly_production.index, y=hourly_production, mode='lines', name='Hourly Production'))
fig.update_layout(title_text="Hourly Production of Photovoltaic System")
fig.update_xaxes(title_text="Time")
fig.update_yaxes(title_text="Power (W)")

# Show the Figure object
fig.show()

In [ ]:
# Calculate the monthly production of the photovoltaic system
monthly_production = hourly_production.resample('M').sum()
total_monthly_production = pd.DataFrame({'Month': monthly_production.index.strftime('%B'),
                                        'Monthly Solar Energy Production (kWh)': monthly_production.values})

# Create a Figure object for the monthly production
fig2 = go.Figure()
fig2.add_trace(go.Bar(
                    x=total_monthly_production['Month'],
                    y=total_monthly_production['Monthly Solar Energy Production (kWh)'],
                    name="Monthly Solar Energy Production (kWh)",
                    marker_color='blue'
))
fig2.update_layout(title_text='Monthly Solar Energy Production (kWh)',
                  xaxis=dict(type='category'))

# Show the Figure object
fig2.show()

In [ ]:
# Calculate the annual energy production
annual_energy_production = hourly_production.sum()

# Calculate the capacity factor
capacity_factor = annual_energy_production / (pr * 8760)

# Define the desired energy demand
energy_demand = 100000  # kWh/year

# Calculate the system size required to meet the desired energy demand
system_size = energy_demand / capacity_factor

# Define the initial investment cost, operating and maintenance costs, and expected lifetime of the system
initial_investment_cost = 1000  # $/kW
operating_and_maintenance_costs = 0.01  # $/kWh
expected_lifetime = 25  # years

# Calculate the levelized cost of energy (LCOE)
discount_rate = 0.05
lcoe = (initial_investment_cost * system_size + operating_and_maintenance_costs * energy_demand * expected_lifetime) / (energy_demand * expected_lifetime * (1 - (1 + discount_rate) ** -expected_lifetime))

# Calculate the payback period
payback_period = initial_investment_cost * system_size / (annual_energy_production * discount_rate)

# Print the results
print(f"Annual energy production: {annual_energy_production:.2f} kWh")
print(f"Capacity factor: {capacity_factor:.2f}")
print(f"System size: {system_size:.2f} kW")
print(f"Levelized cost of energy (LCOE): ${lcoe:.2f}/kWh")
print(f"Payback period: {payback_period:.2f} years")

Annual energy production: 224046.21 kWh
Capacity factor: 0.10
System size: 977476.92 kW
Levelized cost of energy (LCOE): $554.85/kWh
Payback period: 87256.72 years


In [ ]:
import pandas as pd
import numpy as np
# Calculate the levelized cost of energy (LCOE) for a range of system sizes
system_sizes = np.linspace(1, 100, 100)
lcoes = (initial_investment_cost * system_sizes + operating_and_maintenance_costs * energy_demand * expected_lifetime) / (energy_demand * expected_lifetime * (1 - (1 + discount_rate) ** -expected_lifetime))


In [ ]:
# Create a line chart of the levelized cost of energy (LCOE) over a range of system sizes
fig2 = go.Figure()
fig2.add_trace(go.Scatter(
                    x=system_sizes,
                    y=lcoes,
                    name="Levelized Cost of Energy (LCOE) ($/kWh)",
                    line=dict(color='red')
))
fig2.update_layout(title_text='Levelized Cost of Energy (LCOE) ($/kWh) vs. System Size (kW)',
                  xaxis=dict(title='System Size (kW)'),
                  yaxis=dict(title='Levelized Cost of Energy (LCOE) ($/kWh)'))

# Show the figures
fig2.show()

In [ ]:
# Calculate the relative and absolute airmass
from pvlib import atmosphere
airmass_relative = atmosphere.get_relative_airmass(solar_position['apparent_zenith'])
airmass_absolute = atmosphere.get_absolute_airmass(airmass_relative)
# Create a line graph of the relative and absolute airmass
fig = go.Figure()
fig.add_trace(go.Scatter(
                    x=times,
                    y=airmass_relative,
                    name="Relative Air Mass",
                    line=dict(color='blue')
))
fig.add_trace(go.Scatter(
                    x=times,
                    y=airmass_absolute,
                    name="Absolute Air Mass",
                    line=dict(color='red')
))
fig.update_layout(title_text='Relative and Absolute Air Mass',
                  xaxis=dict(title='Time'),
                  yaxis=dict(title='Air Mass'))

# Show the figure
fig.show()

In [ ]:
# Create a DatetimeIndex for a specific date and time range
start_date = pd.Timestamp('2022-01-01', tz=tz_str)
end_date = pd.Timestamp('2022-12-31', tz=tz_str)
dates = pd.date_range(start=start_date, end=end_date, freq='D')

# Calculate the sunrise, sunset, and transit times for each date in the DatetimeIndex
sun_times = site.get_sun_rise_set_transit(dates)

# Create a line graph of the sunrise, sunset, and transit times
fig = go.Figure()
fig.add_trace(go.Scatter(
                    x=sun_times.index,
                    y=sun_times['sunrise'].dt.hour + sun_times['sunrise'].dt.minute / 60,
                    name="Sunrise",
                    line=dict(color='blue')
))
fig.add_trace(go.Scatter(
                    x=sun_times.index,
                    y=sun_times['sunset'].dt.hour + sun_times['sunset'].dt.minute / 60,
                    name="Sunset",
                    line=dict(color='red')
))
fig.add_trace(go.Scatter(
                    x=sun_times.index,
                    y=sun_times['transit'].dt.hour + sun_times['transit'].dt.minute / 60,
                    name="Transit",
                    line=dict(color='green')
))
fig.update_layout(title_text='Sunrise, Sunset, and Transit Times',
                  xaxis=dict(title='Date'),
                  yaxis=dict(title='Time (hours)'))

# Show the figure
fig.show()
